In [ ]:
%%capture

## Requires pip install the following packages
## - openai
## - tabulate
%pip install --upgrade openai tabulate

## Helper functions

### Function to display CSV has html

In [ ]:
import csv
from tabulate import tabulate
from io import StringIO
from IPython.display import display, HTML

def display_csv(title, csv_str):
    csv_reader = csv.reader(StringIO(csv_str), delimiter="|")
    html_out = tabulate(csv_reader, headers="firstrow", tablefmt="html")
    display(HTML("<h2>{}</h2>{}".format(title, html_out)))
    
def extract_title(response):
    return response.content.split("\n")[-1].split("|")[0].strip()


## API CALLS

### Configure api key and test setup

In [ ]:
import os
import openai
openai.organization = os.getenv('BRAINSTORMER_OPENAI_ORG')
openai.api_key = os.getenv('BRAINSTORMER_OPENAI_KEY')

In [ ]:
system_prompt = """
You're a brainstorming application.

It's your job to suggest ideas from the promps the user will send you.

Your output will always be a well formed CSV document. Where the columns will be separated by the character |.

The document will have a list of ideas and every idea will have the columns "summary" with the idea summary and "description" with the idea description.

For example with the prompt: "Activities with children for a rainy day" an example output could be:

```
summary | description
Indoor Treasure Hunt | Create a treasure hunt with clues and small prizes hidden throughout the house. Children will have a great time searching for the hidden treasure and solving the clues.
DIY Playdough | Make homemade playdough with flour, salt, water, and food coloring. Kids will love mixing the ingredients together and then playing with the colorful playdough.
Movie Marathon | Choose a few of your favorite movies and have a movie marathon with the kids. Don't forget the popcorn and snacks!
```

The summary field cannot exceed 4 words and the description can be as long as necessary.
"""

### Prompt: Ideas for a tabletop card game

In [ ]:
topic="Ideas for a tabletop card game"

In [ ]:
PARAMS = {
    "iterations": 2,
    "temperature": 1, ## Higher=+Random Default: 1
    "top_p": 1, ## Less=+Random. Default 1
    "presence_penalty": 0, ## Default 0
    "frequency_penalty": 0, ##  Default 0
}

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": topic}
]
title=topic
for i in range(PARAMS["iterations"]):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        presence_penalty=PARAMS["presence_penalty"],
        frequency_penalty=PARAMS["frequency_penalty"],
        temperature=PARAMS["temperature"],
        top_p=PARAMS["top_p"],
        messages=messages)
    response=completion.choices[0].message
    next_message = "More like \"{}\"".format(extract_title(response))
    messages.append(dict(response))
    messages.append({"role": "user", "content": next_message})
    display_csv(title, response.content)
    title=extract_title(response)


In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": topic}
]
title=topic
for i in range(PARAMS["iterations"]):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        presence_penalty=PARAMS["presence_penalty"],
        frequency_penalty=PARAMS["frequency_penalty"],
        temperature=PARAMS["temperature"],
        top_p=PARAMS["top_p"],
        messages=messages)
    response=completion.choices[0].message
    next_message = "More like \"{}\"".format(extract_title(response))
    messages.append(dict(response))
    messages.append({"role": "user", "content": next_message})
    display_csv(title, response.content)
    title=extract_title(response)
